# Character-wise results

In [2]:
import json
from collections import defaultdict
from pathlib import Path

import pandas as pd

pd.options.plotting.backend = "plotly"

character_data = defaultdict(list)
# level = "page_level"
level = "line_level"

for model_dir in Path(f"../../output/valset_evaluation/{level}/").iterdir():
    model_name = model_dir.name
    collection_level = json.loads((model_dir / "all_rows.json").read_text())

    for character, score_dict in collection_level.items():
        if type(score_dict) is not dict:
            continue
        character_data["model"].append(model_name)
        character_data["F1_score"].append(score_dict["F1_mean"])
        character_data["character"].append(character)

char_df = pd.DataFrame(character_data)

In [ ]:
import dash
import plotly.express as px
from dash import Input, Output, dcc, html

app = dash.Dash(__name__)

app.layout = html.Div(
    [
        dcc.Dropdown(
            id="character-dropdown",
            options=char_df.character.unique(),
            value="Ä",
        ),
        dcc.Graph(id="bar-chart"),
    ]
)


@app.callback(Output("bar-chart", "figure"), [Input("character-dropdown", "value")])
def update_bar_chart(selected_character):  # noqa
    filtered_df = char_df[char_df.character == selected_character]
    filtered_df = filtered_df.sort_values("F1_score")
    fig = px.bar(
        filtered_df,
        x="model",
        y="F1_score",
        barmode="group",
        title=selected_character,
    )
    return fig


if __name__ == "__main__":
    app.run_server(debug=True)